In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
#dependencies
!pip install contractions
import warnings
warnings.filterwarnings("ignore")
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('words')
import re
import pickle
import numpy as np
import pandas as pd
import contractions
import seaborn as sns
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, Conv1D, MaxPooling1D, Dropout, BatchNormalization
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
import gensim
from sklearn.metrics import confusion_matrix
import tensorflow  as tf 
from tensorflow.keras.models import Model, load_model
#from tensorflow.keras.callbacks import ReduceLROnPlateau, LearningRateScheduler, EarlyStopping, ModelCheckpoint
from kaggle_datasets import KaggleDatasets
import transformers
from transformers import TFAutoModel, AutoTokenizer
from tqdm.notebook import tqdm
from tokenizers import Tokenizer, models, pre_tokenizers, decoders, processors
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.optimizers import Adam
import tensorflow.keras


In [3]:
df_2 = pd.read_csv("/kaggle/input/fake-news/train.csv", header=0, index_col=0)
df_t = pd.read_csv("/kaggle/input/fake-news/test.csv", header=0, index_col=0)
df_2 = df_2.drop(['title','author'], axis = 1)
df_t = df_t.drop(['title','author'], axis = 1)
df_2.dropna(inplace = True)
df_t.fillna('',inplace = True)
#print(df_2.isnull().sum(axis = 0))

In [4]:
#functions
def regular_encode(texts, tokenizer, maxlen = 512 ):
    enc_di = tokenizer.batch_encode_plus(texts,return_attention_mask = False, return_token_type_ids  = False,
                                        pad_to_max_length = True, max_length = maxlen, truncation = True)
    return np.array(enc_di["input_ids"])

In [5]:
def build_model(transformer,maxlen = 512):
    input_word_ids = Input(shape=(maxlen,), dtype=tf.int32, name="input_word_ids")
    sequence_output = transformer(input_word_ids)[0]
    cls_token = sequence_output[:, 0, :]
    out = Dense(1, activation='sigmoid')(cls_token)
    
    model = Model(inputs=input_word_ids, outputs=out)
    model.compile(Adam(lr=1e-5), loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [6]:
import tensorflow as tf
# Detect hardware, return appropriate distribution strategy
try:
    # TPU detection. No parameters necessary if TPU_NAME environment variable is
    # set: this is always the case on Kaggle.
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    # Default distribution strategy in Tensorflow. Works on CPU and single GPU.
    strategy = tf.distribute.get_strategy()
print("REPLICAS: ", strategy.num_replicas_in_sync)


In [7]:
from kaggle_datasets import KaggleDatasets
AUTO = tf.data.experimental.AUTOTUNE

# Data access
GCS_DS_PATH = KaggleDatasets().get_gcs_path()

# Configuration
EPOCHS = 2
BATCH_SIZE = 16 * strategy.num_replicas_in_sync
MAX_LEN = 152
MODEL = 'jplu/tf-xlm-roberta-large'

In [8]:
#from transformers import TFAutoModel, AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL)

In [9]:
train = df_2[['text', 'label']]
    

In [10]:
x_train = regular_encode(list(train.text.values), tokenizer, maxlen=MAX_LEN)
#x_valid = regular_encode(list(x_text.text.values), tokenizer, maxlen=MAX_LEN)
x_test = regular_encode(list(df_t.text.values), tokenizer, maxlen=MAX_LEN)

y_train = train.label.values
#y_valid = val_data.toxic.values

In [11]:
train_dataset = (
    tf.data.Dataset
    .from_tensor_slices((x_train, y_train))
    .repeat()
    .shuffle(2048)
    .batch(BATCH_SIZE)
    .prefetch(AUTO)
    
)
test_dataset = (
    tf.data.Dataset
    .from_tensor_slices(x_test)
    .batch(BATCH_SIZE)
)

In [12]:
%%time
with strategy.scope():
    transformer_layer = TFAutoModel.from_pretrained(MODEL)
    model = build_model(transformer_layer, maxlen=MAX_LEN)
model.summary()

In [13]:
n_steps = x_train.shape[0] // BATCH_SIZE
train_history = model.fit(
    train_dataset,
    steps_per_epoch=n_steps,
    epochs=EPOCHS
)

In [14]:
y_t = model.predict(x_test, verbose=1)
y_t = (y_t >= 0.5).astype("int")
result = pd.DataFrame({"id" :df_t.index, "label":y_t.squeeze() }, index = None )
result.to_csv("result_transformer.csv",index = False)